# Generative AI Assignment — Embeddings & Vector Stores
**Course:** Gen AI  
**Author:** Mehdy Mokhtari
**Date:** 1/8/1404

---

## 📘 Introduction

In this part of the assignment, we focus on **embeddings** — a fundamental concept in Generative AI that allows text data to be converted into numerical representations (vectors). These embeddings help models understand **semantic meaning**, enabling **similarity search**, **information retrieval**, and **contextual understanding**.

You will learn how to generate embeddings locally using a **fine-tuned Persian BERT model (ParsBERT)** and store them efficiently in a **FAISS vector store** for semantic search tasks.

---

## 🧩 What You’ll Learn

- The concept and purpose of **text embeddings**.  
- How to **install and use LangChain with HuggingFace** for embedding generation.  
- How to use **ParsBERT (HooshvareLab/bert-base-parsbert-uncased)** for Persian text embeddings.  
- How to **build and test a FAISS vector store** to perform efficient similarity searches.  
- How to work with embeddings **locally**, without relying on external APIs like OpenAI.

## 1.Download Embedding Model

In [5]:
# !pip install langchain langchain-huggingface langchain_community faiss-cpu

INFO: pip is looking at multiple versions of langchain-huggingface to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests

In [1]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [2]:
model_name = "HooshvareLab/bert-base-parsbert-uncased"

model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

# Embedding model (ParsBERT)
hf_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/654M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [3]:
# Example data (Persian text)
text = """ ما در هوشواره معتقدیم با انتقال صحیح دانش و آگاهی همه افراد میتوانند از ابزار های هوشمند استفاده بکنند. شعار ما هوش مصنوعی برای همه است.

"""
embed = hf_embedding.embed_query(text)
print(len(embed))

768


## 2. Faiss Vector Store

In [4]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
import faiss
import os

In [5]:
# FAISS index (L2) with correct dimension
index = faiss.IndexFlatL2(len(hf_embedding.embed_query("سلام")))

# Corpus
texts = [
    "بردارهای معنایی نمایش عددی از متن هستند.",
    "FAISS برای جستجوی شباهت روی بردارها استفاده می‌شود.",
    "مدل‌های زبانی در خلاصه‌سازی و بازیابی اطلاعات مفیدند."
]

# Build vector store
vDB = FAISS.from_texts(texts, hf_embedding)


# Query
query = "چطور جستجوی معنایی انجام دهیم؟"
for i, d in enumerate(vDB.similarity_search(query, k=3), 1):
    print(f"{i}. {d.page_content}")

1. مدل‌های زبانی در خلاصه‌سازی و بازیابی اطلاعات مفیدند.
2. FAISS برای جستجوی شباهت روی بردارها استفاده می‌شود.
3. بردارهای معنایی نمایش عددی از متن هستند.
